# Tutorial: Plan-and-Act Module Inspection and Real Tool Demo

Audience:
- AI Researchers
- Data Scientists
- Agent Architects / Agent Scientists

Prerequisites:
- Python 3.11+
- Environment installed from this repo (`pip install -e .[dev]`)
- Optional: `OPENAI_API_KEY` in `.env` to run GPT-4 path

Learning goals:
1. Import and inspect each core module in the scaffold.
2. Understand Planner/Executor/Replanner interfaces.
3. Run a complete local workflow episode.
4. Run a real tool-integrated example using GitHub API.

Navigation links:
- [Reading guide](../READING_GUIDE.md)
- [Project README](../README.md)
- [Reproduction plan](../REPRODUCTION_PLAN.md)
- [Paper review](../PLAN_AND_ACT_review.md)



## Outline

1. Setup and imports
2. Inspect core modules (`types`, `schemas`, `state`)
3. Inspect agent modules (`planner`, `executor`, `replanner`, `judge`)
4. Inspect and run graph workflow (`workflow`, `transitions`)
5. Real tool demo: GitHub API integration
6. Tool-augmented mini Plan-and-Act episode


In [ ]:
from __future__ import annotations

import inspect
import importlib
import os
import sys
from pathlib import Path

from dotenv import load_dotenv

# Resolve project root no matter where this notebook is launched from.
cwd = Path.cwd().resolve()
PROJECT_ROOT = cwd if (cwd / "src").exists() else cwd.parent
if not (PROJECT_ROOT / "src").exists():
    raise RuntimeError("Could not locate project root with src/ directory")

if str(PROJECT_ROOT / "src") not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT / "src"))

load_dotenv(PROJECT_ROOT / ".env")

print("Project root:", PROJECT_ROOT)
print("OPENAI_API_KEY detected:", bool(os.getenv("OPENAI_API_KEY", "").strip()))
print("OPENAI_MODEL:", os.getenv("OPENAI_MODEL", "(default in config)"))


## Step 1 - Inspect Core Modules

We inspect `plan_and_act.core` to understand the data contracts that keep the system clean and testable.


In [ ]:
core_modules = [
    "plan_and_act.core.types",
    "plan_and_act.core.schemas",
    "plan_and_act.core.state",
]

for module_name in core_modules:
    mod = importlib.import_module(module_name)
    print(f"\n=== {module_name} ===")
    symbols = [n for n in dir(mod) if not n.startswith("_")]
    print("symbols:", symbols)

from plan_and_act.core.schemas import PlanStep, PlannerOutput, ExecutorAction
from plan_and_act.core.state import build_initial_state

print("\nPlanStep signature:", inspect.signature(PlanStep))
print("PlannerOutput fields:", PlannerOutput.model_fields.keys())
print("ExecutorAction fields:", ExecutorAction.model_fields.keys())

example_state = build_initial_state(
    goal="Find top contributor of a GitHub repo",
    max_steps=5,
    dynamic_replanning=True,
    use_cot=False,
)
print("\nInitial state keys:", sorted(example_state.keys()))


## Step 2 - Inspect Agent Modules

Now we inspect each agent class and run a small heuristic pass.


In [ ]:
from plan_and_act.core.types import ModelConfig
from plan_and_act.prompts.templates import PromptTemplates
from plan_and_act.agents.planner import PlannerAgent
from plan_and_act.agents.executor import ExecutorAgent
from plan_and_act.agents.replanner import ReplannerAgent
from plan_and_act.agents.judge import JudgeAgent
from plan_and_act.core.schemas import PlanStep

prompts = PromptTemplates(config_dir=str(PROJECT_ROOT / "configs" / "prompts"))

# Use OpenAI path when key exists, else deterministic heuristic path.
provider = "openai" if os.getenv("OPENAI_API_KEY", "").strip() else "heuristic"
mcfg = ModelConfig(provider=provider, model=os.getenv("OPENAI_MODEL", "gpt-4"), temperature=0.0)

planner = PlannerAgent(mcfg, prompts)
executor = ExecutorAgent(mcfg, prompts)
replanner = ReplannerAgent(mcfg, prompts)
judge = JudgeAgent()

print("PlannerAgent.plan signature:", inspect.signature(planner.plan))
print("ExecutorAgent.act signature:", inspect.signature(executor.act))
print("ReplannerAgent.replan signature:", inspect.signature(replanner.replan))

plan = planner.plan(
    goal="Find the top contributor and return profile link",
    observation="Tooling available: GitHub API",
    action_history=[],
    use_cot=False,
)
print("\nPlanner output:")
print(plan.model_dump())

action = executor.act(
    goal=plan.goal,
    current_step=plan.steps[0] if plan.steps else PlanStep(step_id=1, intent="Fallback", success_criteria=""),
    observation="No environment action has been executed yet.",
    step_index=0,
    total_steps=max(len(plan.steps), 1),
    use_cot=False,
)
print("\nExecutor output:")
print(action.model_dump())


## Step 3 - Inspect and Run Graph Workflow

This executes the current LangGraph workflow (`Planner -> Executor -> Replanner`) with the local simulator adapter.


In [ ]:
from plan_and_act.graph.workflow import build_workflow
from plan_and_act.eval.metrics import compute_episode_metrics

workflow = build_workflow(planner, executor, replanner)
state = build_initial_state(
    goal="Follow the top contributor of this GitHub project",
    max_steps=4,
    dynamic_replanning=True,
    use_cot=False,
)

final_state = workflow.invoke(state)
metrics = compute_episode_metrics(final_state)

print("Final state summary:")
print({
    "success": final_state["success"],
    "step_count": final_state["step_count"],
    "final_answer": final_state["final_answer"],
    "replans": sum(1 for n in final_state["notes"] if "Replanned" in n),
})
print("\nMetrics:", metrics)


## Step 4 - Real Tool Demo (GitHub API)

Below is a **real external tool** call. We query GitHub REST API to fetch the top contributor of a real repository.


In [ ]:
import json
import urllib.request
from dataclasses import dataclass


@dataclass
class GitHubTool:
    base_url: str = "https://api.github.com"

    def _get(self, path: str) -> dict | list:
        req = urllib.request.Request(
            url=f"{self.base_url}{path}",
            headers={"User-Agent": "plan-and-act-repro-demo"},
            method="GET",
        )
        with urllib.request.urlopen(req, timeout=20) as resp:
            payload = resp.read().decode("utf-8")
        return json.loads(payload)

    def top_contributor(self, owner: str, repo: str) -> dict:
        data = self._get(f"/repos/{owner}/{repo}/contributors?per_page=1")
        if not data:
            return {"found": False, "reason": "No contributors returned"}
        top = data[0]
        return {
            "found": True,
            "owner": owner,
            "repo": repo,
            "login": top.get("login", ""),
            "contributions": top.get("contributions", 0),
            "profile_url": top.get("html_url", ""),
        }


gh_tool = GitHubTool()
result = gh_tool.top_contributor("openai", "openai-python")
result


## Step 5 - Tool-Augmented Mini Episode

We combine planner + real tool observation + replanner/executor to simulate a realistic planning loop.


In [ ]:
goal = "Find the top contributor of openai/openai-python and provide profile URL"

initial_plan = planner.plan(
    goal=goal,
    observation="Tool available: GitHub REST API contributor endpoint",
    action_history=[],
    use_cot=False,
)

tool_observation = (
    f"Top contributor for openai/openai-python: {result.get('login')} "
    f"with {result.get('contributions')} contributions, profile={result.get('profile_url')}"
)

updated_plan = replanner.replan(
    goal=goal,
    previous_plan=[s.model_dump() for s in initial_plan.steps],
    action_history=[{"tool": "github_api", "observation": result}],
    observation=tool_observation,
    use_cot=False,
)

# Ask executor to produce one final action based on the replanned context.
final_step = updated_plan.steps[0] if updated_plan.steps else PlanStep(step_id=1, intent="Return final answer", success_criteria="")
final_action = executor.act(
    goal=goal,
    current_step=final_step,
    observation=tool_observation,
    step_index=0,
    total_steps=1,
    use_cot=False,
)

summary = {
    "goal": goal,
    "initial_plan_steps": [s.intent for s in initial_plan.steps],
    "tool_result": result,
    "updated_plan_steps": [s.intent for s in updated_plan.steps],
    "final_action": final_action.model_dump(),
}
summary


## Next Steps

1. Replace `GitHubTool` with browser tools (Playwright/WebArena adapters).
2. Attach run logging for each notebook episode into `artifacts/runs/`.
3. Compare heuristic vs GPT-4 mode on identical goals and collect metrics.
